In [10]:
from os import environ, listdir, path

environ["HF_HOME"] = "/oak/stanford/groups/mrivas/projects/multiomics/tlmenest/LLMChemCreator/huggingface"

In [11]:
from transformers import AutoModel, AutoTokenizer
from torch        import no_grad, save, load
from Bio.PDB      import PDBParser
from Bio.SeqUtils import seq1

In [12]:
model_name = "facebook/esm2_t33_650M_UR50D"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Define your PDB ID and file path
PDB_ID = "YOUR_PDB_ID_HERE" 
PDB_file_path = "proteins/apol1.pdb"
pdbparser = PDBParser()

structure = pdbparser.get_structure(PDB_ID, PDB_file_path)
chains = {chain.id: seq1(''.join(residue.resname.strip() for residue in chain if residue.id[0] == " " and residue.resname.isalpha()), undef_code="X") for chain in structure.get_chains()}

In [14]:
chain = chains['A']

In [15]:
pdb_file_path = "proteins/apol1.pdb"
sequence = extract_sequence_from_pdb(pdb_file_path)

NameError: name 'extract_sequence_from_pdb' is not defined

In [ ]:
inputs = tokenizer(chain, return_tensors="pt")

with no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state  

In [ ]:
embeddings.shape

In [ ]:
embeddings

In [16]:
protein_files = listdir('proteins')
pdb_files     = [file for file in protein_files if file.endswith('.pdb')]

In [20]:
protein_files

['dnmt3a.pdb',
 'tbc1d1.pdb',
 'ankh.pdb',
 'scn10a.pdb',
 '.ipynb_checkpoints',
 'gpr75.pdb',
 'embeddings',
 'gipr.pdb',
 'umod.pdb',
 'cfhr5.pdb',
 'pcsk9.pdb',
 'rnf186.pdb',
 'tyk2.pdb',
 'nlrp3.pdb',
 'il4r.pdb',
 'dpp9.pdb',
 'apol1.pdb',
 'scn9a.pdb',
 'f10a.pdb',
 'slc6a19.pdb',
 'card6.pdb',
 'tnfsf15.pdb',
 'gys1.pdb',
 'dpp4.pdb',
 'adcy5.pdb',
 'map3k15.pdb',
 'ifih1.pdb',
 'slc30a8.pdb',
 'ubash3a.pdb',
 'lrp2.pdb']

In [17]:
pdbparser = PDBParser()
chains    = []

for file in pdb_files:

    PDB_file_path = path.join('proteins', file)
    structure = pdbparser.get_structure(PDB_ID, PDB_file_path)
    output    = {chain.id: seq1(''.join(residue.resname.strip() for residue in chain if residue.id[0] == " " and residue.resname.isalpha()), undef_code="X") for chain in structure.get_chains()}

    chain = output['A']
    (chains.append(chain))

In [18]:
for chain in chains:

    print(len(chain))

912
1168
492
1956
540
466
640
569
692
227
1187
1036
825
863
398
1988
488
634
1037
251
737
766
1261
1313
921
52
451
1225


In [19]:
%%time

embeddings = []

for chain in chains:

    inputs = tokenizer(chain, return_tensors="pt")

    with no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state  
        print('Done')
    embeddings.append(embedding)

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
CPU times: user 8min 40s, sys: 1min 55s, total: 10min 35s
Wall time: 11min


In [ ]:
embeddings

In [21]:
for i in range(len(pdb_files)):

    save_path = path.join('proteins/embeddings/', (pdb_files[i][:-4] + '_embedding.pt'))
    save(embeddings[i], save_path)

In [22]:
loaded_tensor = load('proteins/embeddings/apol1_embedding.pt')

# Display the loaded tensor
loaded_tensor.shape

torch.Size([1, 400, 1280])

In [ ]:
def canonicalize(s):
    return Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True, isomericSmiles=False)

def batch_split(data, batch_size=64):
    for i in range(0, len(data), batch_size):
        yield data[i:i+batch_size]

def embed_and_save(model, df, tokenizer, batch_size=64):
    model.eval()
    for batch in batch_split(df, batch_size):
        smiles = batch['canonical_smiles'].apply(canonicalize)
        batch_enc = tokenizer.batch_encode_plus(smiles.tolist(), padding=True, add_special_tokens=True, return_tensors='pt')
        idx, mask = batch_enc['input_ids'], batch_enc['attention_mask']
        with torch.no_grad():
            token_embeddings = model(idx, attention_mask=mask).last_hidden_state
        input_mask_expanded = mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        embeddings = sum_embeddings / sum_mask
        for i, chembl_id in enumerate(batch['chembl_id']):
            # Ensure the directory exists
            os.makedirs('ligands', exist_ok=True)
            # Save each tensor
            torch.save(embeddings[i].cpu(), f'ligands/{chembl_id}.pt')

In [6]:
from os import listdir

In [7]:
embs = listdir('ligands/')

In [9]:
len(embs)

869510